In [1]:
import pandas as pd
import plotly.express as px

car_data = pd.read_csv('../vehicles.csv') # lendo os dados

###
Analisando o DF

In [2]:
car_data.describe()

,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,47906.000000,46265.000000,43633.000000,25572.0,51525.00000
mean,12132.464920,2009.750470,6.125235,115553.461738,1.0,39.55476
std,10040.803015,6.282065,1.660360,65094.611341,0.0,28.20427
min,1.000000,1908.000000,3.000000,0.000000,1.0,0.00000
25%,5000.000000,2006.000000,4.000000,70000.000000,1.0,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,1.0,33.00000
75%,16839.000000,2014.000000,8.000000,155000.000000,1.0,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.0,271.00000


In [3]:
car_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [4]:
car_data.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [5]:
car_data.duplicated().sum()

0

In [6]:
car_data.corr(numeric_only=True) # parametro para apenas aceitar colunas numéricas

,price,model_year,cylinders,odometer,is_4wd,days_listed
price,1.000000,0.428656,0.296627,-0.420299,NaN,-0.000682
model_year,0.428656,1.000000,-0.148040,-0.473948,NaN,-0.005703
cylinders,0.296627,-0.148040,1.000000,0.090160,NaN,0.005517
odometer,-0.420299,-0.473948,0.090160,1.000000,NaN,0.003804
is_4wd,NaN,NaN,NaN,NaN,NaN,NaN
days_listed,-0.000682,-0.005703,0.005517,0.003804,NaN,1.000000


In [7]:
car_data.isna().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

# VALORES AUSENTES
1. trocar os valores ausentes em **is_4wd** para 0 (ou seja, é uma coluna booleana sendo 1 = 4x4 e 0 = tração em 2 rodas)
2. preencher os valores ausentes em **model_year** de acordo com odômetro e preço
3. preencher os valores ausentes em **odometer** de acordo com o ano do modelo e condição
4. preencher os valores ausentes em **paint_color** com o nome "desconhecido"

In [8]:
car_data['is_4wd'] = car_data['is_4wd'].fillna('0.0')

In [9]:
car_data.loc[car_data['model_year'].isna()].head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
20,6990,NaN,chevrolet tahoe,excellent,8.0,gas,147485.0,automatic,SUV,silver,1.0,2018-08-05,28
65,12800,NaN,ford f-150,excellent,6.0,gas,108500.0,automatic,pickup,white,0.0,2018-09-23,15
69,7800,NaN,ford f-150,like new,8.0,gas,97510.0,automatic,truck,white,1.0,2019-02-20,39
72,3650,NaN,subaru impreza,excellent,NaN,gas,74000.0,automatic,sedan,blue,1.0,2018-08-07,60


In [10]:
car_data.loc[car_data['odometer'] >= 200000.0].median(numeric_only=True) 
#a mediana de do ano de carros que tem mais de 200k de km rodados é 2004

price            4500.0
model_year       2004.0
cylinders           8.0
odometer       226998.0
days_listed        33.0
dtype: float64

In [11]:
car_data.loc[
    (car_data['odometer'] > 200000.0) & 
    (car_data['model_year'].isna()) , 'model_year'] = 2004

In [12]:
# repetindo o mesmo processo
car_data.loc[car_data['odometer'] <= 40000].median(numeric_only=True) 

price          19995.0
model_year      2017.0
cylinders          6.0
odometer       21582.0
days_listed       33.0
dtype: float64

In [13]:
car_data.loc[
    (car_data['odometer'] <= 40000.0) & 
    (car_data['model_year'].isna()) , 'model_year'] = 2017.0

In [14]:
car_data.loc[(car_data['odometer'] >= 40000.0) & (car_data['odometer'] <= 100000.0)].median(numeric_only=True) 

price          13597.0
model_year      2014.0
cylinders          6.0
odometer       74520.5
days_listed       33.0
dtype: float64

In [15]:
car_data.loc[
    (car_data['odometer'] >= 40000.0) & (car_data['odometer'] <= 100000.0) & 
    (car_data['model_year'].isna()), 'model_year'] = 2014.0

In [16]:
car_data.loc[(car_data['odometer'] >= 100000) & (car_data['odometer'] <= 200000)].median(numeric_only=True) 

price            6995.0
model_year       2009.0
cylinders           6.0
odometer       139250.0
days_listed        33.0
dtype: float64

In [17]:
car_data.loc[
    (car_data['odometer'] >= 100000) & (car_data['odometer'] <= 200000) & 
    (car_data['model_year'].isna()), 'model_year'] = 2009.0

In [18]:
car_data.groupby('model_year')['price'].median().tail(20)
# de 2000 a 2005 os valores são abaixo de 5000

model_year
2000.0     3500.0
2001.0     3995.0
2002.0     3999.0
2003.0     4488.0
2004.0     4999.0
2005.0     4995.0
2006.0     5900.0
2007.0     6050.0
2008.0     6995.0
2009.0     6950.0
2010.0     7995.0
2011.0     9995.0
2012.0     9995.0
2013.0    11995.0
2014.0    13990.0
2015.0    17700.0
2016.0    18000.0
2017.0    19700.0
2018.0    23800.0
2019.0    23800.0
Name: price, dtype: float64

In [19]:
car_data.loc[
    (car_data['price'] < 5000) & 
    (car_data['model_year'].isna()), 'model_year'] = 2002.0

In [20]:
car_data.loc[
    (car_data['price'] > 10000) & (car_data['price'] < 20000) &
    (car_data['model_year'].isna()), 'model_year'] = 2015.0

In [21]:
car_data.loc[
    (car_data['price'] > 20000) &
    (car_data['model_year'].isna()), 'model_year'] = 2019.0

In [22]:
# começar a preencher a coluna 'odometer'
car_data.groupby('odometer')['model_year'].median()

odometer
0.0         2010.0
1.0         2007.0
2.0         2005.0
3.0         2011.0
5.0         2018.0
             ...  
840000.0    2010.0
866000.0    2014.0
920000.0    1996.0
980000.0    2014.0
990000.0    1988.5
Name: model_year, Length: 17762, dtype: float64

In [23]:
# filtrando os odômetros de 1 a 1000 km e vendo a média do ano de fabricação com base neles
um_mil_km = car_data[(car_data['odometer'] > 0) & (car_data['odometer'] <= 1000)]

media_ano = um_mil_km['model_year'].mean()
media_ano

2008.6605783866057

In [24]:
#car_data[(car_data['odometer'].isna()) & (car_data[car_data['condition'] == 'excelent'])]

In [25]:
car_data[(car_data['odometer'].isna()) & (car_data['model_year'] == 2019.0)]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
125,22900,2019.0,nissan frontier crew cab sv,good,6.0,gas,NaN,other,pickup,blue,1.0,2019-04-15,67
159,23300,2019.0,nissan frontier crew cab sv,good,NaN,gas,NaN,other,pickup,grey,1.0,2018-07-24,73
586,26000,2019.0,toyota rav4,like new,4.0,gas,NaN,automatic,SUV,NaN,0.0,2018-08-09,29
843,22300,2019.0,nissan frontier crew cab sv,good,6.0,gas,NaN,other,pickup,black,1.0,2019-01-22,32
1232,28990,2019.0,ford f-350 sd,good,8.0,diesel,NaN,automatic,truck,white,1.0,2018-11-15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46633,29995,2019.0,ford f-150,excellent,8.0,gas,NaN,automatic,truck,blue,1.0,2019-04-02,55
47188,25200,2019.0,chevrolet silverado 1500 crew,good,8.0,gas,NaN,automatic,pickup,silver,1.0,2018-07-08,41
47392,25800,2019.0,chevrolet silverado 1500 crew,good,8.0,gas,NaN,automatic,pickup,black,1.0,2018-09-19,70
47983,32500,2019.0,chevrolet silverado 3500hd,excellent,8.0,diesel,NaN,automatic,truck,black,1.0,2019-03-23,40


In [26]:
car_data.loc[car_data['model'] == 'honda pilot'].groupby('cylinders').agg('count')

,price,model_year,model,condition,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
cylinders,,,,,,,,,,,,
4.0,2,2,2,2,2,2,2,2,2,2,2,2
5.0,1,1,1,1,1,1,1,1,1,1,1,1
6.0,269,269,269,269,269,225,269,269,225,269,269,269
8.0,2,2,2,2,2,2,2,2,2,2,2,2


In [27]:
# preenchendo a coluna paint_color
car_data['paint_color'] = car_data['paint_color'].fillna('desconhecido')

# TIPOS DE COLUNAS
1. trocar a coluna **model_year** de float para **int**
2. trocar a coluna **cylinders** de float para **int**
3. trocar a coluna **odometer** de float para **int**
4. trocar a coluna **is_4wd** de float para **int**
5. trocar a coluna **date_posted** de object para **datetime**  

# convertendo as colunas de float para int
car_data['model_year'].astype('int')
car_data['cylinders'].astype('int')
car_data['odometer'].astype('int')
car_data['is_4wd'].astype('int')

# convertendo para datetime
car_data['date_posted'] = pd.to_datetime(car_data['date_posted'])

In [28]:
car_data['cylinders'].value_counts()


cylinders
8.0     15844
6.0     15700
4.0     13864
10.0      549
5.0       272
3.0        34
12.0        2
Name: count, dtype: int64

In [29]:
car_data['cylinders'].isna().sum()

5260

In [30]:
car_data.isna().sum()

price              0
model_year       171
model              0
condition          0
cylinders       5260
fuel               0
odometer        7892
transmission       0
type               0
paint_color        0
is_4wd             0
date_posted        0
days_listed        0
dtype: int64